<h1> 1.	Introduction: Business Problem </h1>

Bangalore (also known as Bengaluru) is one of the fastest growing cities in India and a major IT hub. Bangalore has witnessed phenomenal growth in the past several years and many businesses have thrived. This has attracted related and new businesses to the area. For a business to succeed in Bangalore, it is very important to set it up in a location where it can do well. The analysis performed in this project helps identify such locations. 
Let us take two specific cases and analyze them in this project – 
a.	what is the ideal location for setting up a hotel?
b.	what is the ideal location for setting up a coffee shop?


#### Initial set up
Import all the required libraries. Use Bangalore Neighborhoods data downloaded from Kaggle to get the Geo Coordinates (Longitude and Lattitude data) by neighborhood. 
Import Bangalore Neighborhoods data csv into Notebook 

In [9]:
import os, types
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from sklearn.cluster import KMeans
from botocore.client import Config
import ibm_boto3
import json
#!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import seaborn as sns
import matplotlib.pyplot as plt

<h1>2. Data </h1>

a)	Kaggle data source for Neighborhood data: publicly available Kaggle data source that captures Neighborhood in Bangalore, its longitude and latitude

b)	Foursquare API: is used to extract location information by passing latitude and longitude of the various neighborhoods in Bangalore. Foursquare API provides a rich API to extract different venues in the neighborhood. This data is very important as it forms the basis of analysis for clustering. 

#### Add Bangalore Lattitude and Longitude data
Added Bangalore Lattitude and Longitude data from Kaggle into the Watson Studio as a data asset and imported it

In [10]:
def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_a388439f09504d8f90d98c2f50a58cfc = 'https://s3-api.us-geo.objectstorage.softlayer.net'
else:
    endpoint_a388439f09504d8f90d98c2f50a58cfc = 'https://s3-api.us-geo.objectstorage.service.networklayer.com'

client_a388439f09504d8f90d98c2f50a58cfc = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='aGk2Kol6hJsCprs_gv-kV2FWMaX_BoxujQ90FtSEjXw9',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_a388439f09504d8f90d98c2f50a58cfc)

body = client_a388439f09504d8f90d98c2f50a58cfc.get_object(Bucket='capstoneproject-donotdelete-pr-abs02xjbbmgqrb',Key='blr_neighborhoods.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

bangalore_data = pd.read_csv(body)
bangalore_data.head()

,Unnamed: 0,Neighborhood,Latitude,Longitude
0,0,Agram,45.813177,15.977048
1,1,Amruthahalli,13.066513,77.596624
2,2,Attur,11.663711,78.533551
3,3,Banaswadi,13.014162,77.651854
4,4,Bellandur,58.235358,26.683117


#### Drop the column "Unnamed: 0"

In [11]:
bangalore_data.drop(bangalore_data.columns[bangalore_data.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
bangalore_data.head()

,Neighborhood,Latitude,Longitude
0,Agram,45.813177,15.977048
1,Amruthahalli,13.066513,77.596624
2,Attur,11.663711,78.533551
3,Banaswadi,13.014162,77.651854
4,Bellandur,58.235358,26.683117


#### Use geopy library to get the latitude and longitude values of Bangalore City.

In [12]:
address = 'Bengaluru, Karnataka'
geolocator = Nominatim(user_agent="blr_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geographical coordinate of Bangalore are {}, {}.'.format(latitude, longitude))

The geographical coordinate of Bangalore are 12.9791198, 77.5912997.


#### Create a map of New York with neighborhoods superimposed on top.

In [13]:
# create map of Bangalore using latitude and longitude values
blr_map = folium.Map(location=[latitude, longitude], zoom_start=11)    

# add markers to map
for lat, lng, location in zip(bangalore_data['Latitude'], bangalore_data['Longitude'], bangalore_data['Neighborhood']):
    label = '{}'.format(location)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(blr_map)
    
blr_map

#### Connect to Foursquare API to retrieve location information for the neighborhoods

In [14]:
CLIENT_ID = 'I0TMLC4YZ04IL5VSKGQS12XW2TYGA00MOARR1ECC1NJWX2YC' # your Foursquare ID
CLIENT_SECRET = 'DIEKT3E1WL20SY5B4UMAVZERHAFPIMSI4VIZXG2FLYZJY1DA' # your Foursquare Secret
VERSION = '20210408' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

#### Let's create a function to repeat the same process to all the neighborhoods in Bangalore

In [15]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

<h1> 3.	Methodology </h1>

High-level methodology used for this project is provided below:

a)	Both the data sets mentioned in section #2 are used to extract the nearby venues information for each of the neighborhoods in Bangalore
b)	K-Means clustering is used to perform the analysis and determine the clusters. 
c)	Elbow method is used to find out the right “K” value. Optimal “k” value was determined to be “8” for this analysis.

#### Now write the code to run the above function on each neighborhood and create a new dataframe called bangalore_venues.

In [16]:
bangalore_venues = getNearbyVenues(names=bangalore_data['Neighborhood'],
                                   latitudes=bangalore_data['Latitude'],
                                   longitudes=bangalore_data['Longitude']
                                  )

Agram


KeyError: 'groups'

In [ ]:
print(bangalore_venues.shape)
bangalore_venues.head()

In [ ]:
bangalore_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(bangalore_venues['Venue Category'].unique())))

<h1> 4. Analysis </h1>

Venue data is carefully analyzed with help of bar graphs to visualize the neighborhoods and the number of nearby venues. There are several neighborhoods that have very minimal number of venues.

In [ ]:
df_temp = bangalore_venues.groupby(['Neighborhood'], sort=False)['Venue'].count()
df_temp.plot.bar(figsize=(18,6))

In [ ]:
df10 = df_temp[df_temp >= 10]
df10.plot.bar(figsize=(18,6))

In [ ]:
bangalore_venues_top = bangalore_venues[bangalore_venues['Neighborhood'].isin(df10.index.tolist())]
bangalore_venues_top.head()

In [ ]:
# one hot encoding
bangalore_onehot = pd.get_dummies(bangalore_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
bangalore_onehot['Neighborhood'] = bangalore_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [bangalore_onehot.columns[-1]] + list(bangalore_onehot.columns[:-1])
bangalore_onehot = bangalore_onehot[fixed_columns]

bangalore_onehot.head()

In [ ]:
bangalore_onehot.shape

In [ ]:
bangalore_grouped = bangalore_onehot.groupby('Neighborhood').mean().reset_index()
bangalore_grouped

In [ ]:
num_top_venues = 10

for hood in bangalore_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = bangalore_grouped[bangalore_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = bangalore_grouped['Neighborhood']

for ind in np.arange(bangalore_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(bangalore_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
bangalore_grouped_clustering = bangalore_grouped.drop('Neighborhood', 1)
sum_of_squared_distances = []
K = range(1,15)
for k in K:
    print(k, end=' ')
    kmeans = KMeans(n_clusters=k).fit(bangalore_grouped_clustering)
    sum_of_squared_distances.append(kmeans.inertia_)

In [ ]:
plt.plot(K, sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('sum_of_squared_distances')
plt.title('Elbow Method For Optimal k');

In [ ]:
# set number of clusters
kclusters = 8

bangalore_grouped_clustering = bangalore_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(bangalore_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

bangalore_merged = bangalore_venues[bangalore_venues.columns[0:3]].drop_duplicates()

# merge toronto_grouped with bangalore_grouped to add latitude/longitude for each neighborhood
bangalore_merged = bangalore_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

bangalore_merged.head() # check the last columns!

<h1> 5.	Results and Discussion </h1>

After clustering and segmenting following clusters have been created and analyzed graphically to identify the ideal locations to open:
a)	Hotel 
b)	Coffee Shop
Review below the graphs of the various clustering providing the number of venues by venue name.


In [ ]:
cluster1=bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 0, bangalore_merged.columns[[0] + list(range(4, bangalore_merged.shape[1]))]]

ven_list_1 = (cluster1['1st Most Common Venue'].append(
    cluster1['2nd Most Common Venue']).append(
    cluster1['3rd Most Common Venue']).append(
    cluster1['4th Most Common Venue']).append(
    cluster1['5th Most Common Venue']).append(
    cluster1['6th Most Common Venue']).append(
    cluster1['7th Most Common Venue']).append(
    cluster1['8th Most Common Venue']).append(
    cluster1['9th Most Common Venue']).append(
    cluster1['10th Most Common Venue']))

In [ ]:
cluster2=bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 1, bangalore_merged.columns[[0] + list(range(4, bangalore_merged.shape[1]))]]

ven_list_2 = (cluster2['1st Most Common Venue'].append(
    cluster2['2nd Most Common Venue']).append(
    cluster2['3rd Most Common Venue']).append(
    cluster2['4th Most Common Venue']).append(
    cluster2['5th Most Common Venue']).append(
    cluster2['6th Most Common Venue']).append(
    cluster2['7th Most Common Venue']).append(
    cluster2['8th Most Common Venue']).append(
    cluster2['9th Most Common Venue']).append(
    cluster2['10th Most Common Venue']))

In [ ]:
cluster3=bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 2, bangalore_merged.columns[[0] + list(range(4, bangalore_merged.shape[1]))]]

ven_list_3 = (cluster3['1st Most Common Venue'].append(
    cluster3['2nd Most Common Venue']).append(
    cluster3['3rd Most Common Venue']).append(
    cluster3['4th Most Common Venue']).append(
    cluster3['5th Most Common Venue']).append(
    cluster3['6th Most Common Venue']).append(
    cluster3['7th Most Common Venue']).append(
    cluster3['8th Most Common Venue']).append(
    cluster3['9th Most Common Venue']).append(
    cluster3['10th Most Common Venue']))

In [ ]:
cluster4=bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 3, bangalore_merged.columns[[0] + list(range(4, bangalore_merged.shape[1]))]]

ven_list_4 = (cluster4['1st Most Common Venue'].append(
    cluster4['2nd Most Common Venue']).append(
    cluster4['3rd Most Common Venue']).append(
    cluster4['4th Most Common Venue']).append(
    cluster4['5th Most Common Venue']).append(
    cluster4['6th Most Common Venue']).append(
    cluster4['7th Most Common Venue']).append(
    cluster4['8th Most Common Venue']).append(
    cluster4['9th Most Common Venue']).append(
    cluster4['10th Most Common Venue']))

In [ ]:
cluster5=bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 4, bangalore_merged.columns[[0] + list(range(4, bangalore_merged.shape[1]))]]

ven_list_5 = (cluster5['1st Most Common Venue'].append(
    cluster5['2nd Most Common Venue']).append(
    cluster5['3rd Most Common Venue']).append(
    cluster5['4th Most Common Venue']).append(
    cluster5['5th Most Common Venue']).append(
    cluster5['6th Most Common Venue']).append(
    cluster5['7th Most Common Venue']).append(
    cluster5['8th Most Common Venue']).append(
    cluster5['9th Most Common Venue']).append(
    cluster5['10th Most Common Venue']))

In [ ]:
cluster6=bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 5, bangalore_merged.columns[[0] + list(range(4, bangalore_merged.shape[1]))]]

ven_list_6 = (cluster6['1st Most Common Venue'].append(
    cluster6['2nd Most Common Venue']).append(
    cluster6['3rd Most Common Venue']).append(
    cluster6['4th Most Common Venue']).append(
    cluster6['5th Most Common Venue']).append(
    cluster6['6th Most Common Venue']).append(
    cluster6['7th Most Common Venue']).append(
    cluster6['8th Most Common Venue']).append(
    cluster6['9th Most Common Venue']).append(
    cluster6['10th Most Common Venue']))

In [ ]:
cluster7=bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 6, bangalore_merged.columns[[0] + list(range(4, bangalore_merged.shape[1]))]]

ven_list_7 = (cluster7['1st Most Common Venue'].append(
    cluster7['2nd Most Common Venue']).append(
    cluster7['3rd Most Common Venue']).append(
    cluster7['4th Most Common Venue']).append(
    cluster7['5th Most Common Venue']).append(
    cluster7['6th Most Common Venue']).append(
    cluster7['7th Most Common Venue']).append(
    cluster7['8th Most Common Venue']).append(
    cluster7['9th Most Common Venue']).append(
    cluster7['10th Most Common Venue']))

In [ ]:
cluster8=bangalore_merged.loc[bangalore_merged['Cluster Labels'] == 7, bangalore_merged.columns[[0] + list(range(4, bangalore_merged.shape[1]))]]

ven_list_8 = (cluster8['1st Most Common Venue'].append(
    cluster8['2nd Most Common Venue']).append(
    cluster8['3rd Most Common Venue']).append(
    cluster8['4th Most Common Venue']).append(
    cluster8['5th Most Common Venue']).append(
    cluster8['6th Most Common Venue']).append(
    cluster8['7th Most Common Venue']).append(
    cluster8['8th Most Common Venue']).append(
    cluster8['9th Most Common Venue']).append(
    cluster8['10th Most Common Venue']))

In [ ]:
ven_list_1.value_counts().head(15).plot.bar()

In [ ]:
ven_list_2.value_counts().head(15).plot.bar()

In [ ]:
ven_list_3.value_counts().head(15).plot.bar()

In [ ]:
ven_list_4.value_counts().head(15).plot.bar()

In [ ]:
ven_list_5.value_counts().head(15).plot.bar()

In [ ]:
ven_list_6.value_counts().head(15).plot.bar()

In [ ]:
ven_list_7.value_counts().head(15).plot.bar()

In [ ]:
ven_list_8.value_counts().head(15).plot.bar()

<h1> 6.	Conclusion </h1> 

By carefully analyzing the clusters, we can conclude that:

#### Hotel

Only Cluster 3 has a hotel, so there is an opportunity to set up a hotel in other clusters Cluster 1, Cluster 2, Cluster 4, Cluster 5, Cluster 6, Cluster 7, and Cluster 8. Out of these clusters, only Cluster 7 and Cluster 2 have significant number of other attractions, making them ideal locations for setting up a hotel

#### Coffee Shop
There are no coffee shops in Cluster 1, Cluster 3, Cluster 5, Cluster 6 and Cluster 8, so they could be potential candidates for setting up a coffee shop. Out of them, Cluster 1 is best suitable for setting up a coffee shop because of the proximity to other attractions.
Find below the map listing the different clusters


In [ ]:
bangalore_clusters_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
colors_array = cm.rainbow(np.linspace(0, 1, kclusters))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(bangalore_merged['Neighborhood Latitude'], bangalore_merged['Neighborhood Longitude'], bangalore_merged['Neighborhood'], bangalore_merged['Cluster Labels']):
    label = folium.Popup(star(poi) + ' Cluster ' + str(cluster+1), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(bangalore_clusters_map)
       
bangalore_clusters_map